In [ ]:
import os
import rasterio
import numpy as np
import geopandas as gpd

from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon, box


class SerialTimer():
    
    def __init__(self,
                 aoi=[15.00, 48.00, 15.01, 48.01],
                 raster_dir='/home/volume2/Austria/Timeseries/',
                 polygon_file='/home/volume2/Austria/GroundTruth/invekos_schlaege_polygon.shp',
                 out_dir='/home/mlamarre/Documents/FieldTimeSeries',
                 time_stamps=np.arange(1,61),
                 dimensions=['BS.VH', 'BS.VV']):
        
        self.raster_dir = raster_dir
        self.polygon_file = polygon_file
        self.out_dir = out_dir
        self.aoi = box(minx=aoi[0],miny=aoi[1],maxx=aoi[2],maxy=aoi[3])
        self.time_stamps = time_stamps
        self.dimensions = dimensions
        columns = ['PolyID', ]
        
    def readPolygons(self):
        if os.path.isfile(self.polygon_file):
            print('Reading shape file...')
            self.polygons = gpd.read_file(self.polygon_file)
            self.poly_crs = self.polygons.crs
        else:
            print('ERROR: wrong shape file')
            
    def readRaster(self, time_stamp, dimension):
        filename = '.'.join([self.raster_dir+str(time_stamp),dimension,'tif'])
        if os.path.isfile(filename):
            print('Reading raster file...')
            self.raster = rasterio.open(filename)
            self.rast_crs = self.raster.crs
        else:
            print('ERROR: wrong raster file')
            
    def selectPolygons(self):
        # Transforms AoI to GeoDataFrame to change CRS and then back to Polygon to use within function
        print('Original coordinates: {}'.format(self.aoi.bounds))
        AOI = gpd.GeoDataFrame({'geometry':self.aoi}, index=[0], crs=self.rast_crs)
        AOI = AOI.to_crs(self.poly_crs)
        AOI = box(minx=AOI.bounds.loc[0]['minx'],
                  miny=AOI.bounds.loc[0]['miny'],
                  maxx=AOI.bounds.loc[0]['maxx'],
                  maxy=AOI.bounds.loc[0]['maxy'])
        print('Transformed coordinates: {}'.format(AOI.bounds))
        
        # Iterate over all the polygons and check if the 1st point is within AOI 
        idxs = []
        for index, row in self.polygons.iterrows():
            if index % 2500 == 0:
                print('Scanned {} out of {} fields. ({}%)'.format(index,self.polygons.shape[0],index/self.polygons.shape[0]*100),end='\r')
            if(Point(row.geometry.exterior.coords[0]).within(AOI)):
                idxs.append(index)        
        print('{} fields were selected'.format(len(idxs)))
        
        # Keep the polygons of interest and change their CRS
        self.polys_OI = self.polygons.iloc[idxs]
        self.polys_OI = self.polys_OI.to_crs(self.rast_crs)
        

In [ ]:
s = SerialTimer()

In [ ]:
s.readPolygons()

In [ ]:
s.readRaster(1,'BS.VV')

In [ ]:
s.selectPolygons()

In [11]:
bigby = gpd.GeoDataFrame(columns = s.dimensions)

In [12]:
bigby


,BS.VH,BS.VV


In [ ]:
bib